# TrialDuraPredict Interactive Dashboard

In [37]:
# Create an interactive dashboard
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
import altair as alt
from dash import Dash, Input, Output, callback, dcc, html, dash_table, State
from jupyter_dash import JupyterDash
from vega_datasets import data
import dash_vega_components as dvc
import gdown

### Use `gdown` library to download CSV of study information for use in dashboard and automatically skip if file already exists

In [38]:
input_path = "./study_info_df.csv"

if os.path.isfile(input_path):
    pass
else:
    # Should only take ~30 seconds
    url = "https://drive.google.com/file/d/1sSYhHvWkYVRFoB1ulEblmTwo9iSibYpG/view?usp=drive_link"

    output = "study_info_df.csv"

    gdown.download(url, output, fuzzy=True)

In [39]:
# Read in downlaoded CSV
input_path = "./study_info_df.csv"
study_info_df = pd.read_csv(input_path)
study_info_df

,nctId,status,phases,description,inclusion_criteria,exclusion_criteria,intervention_type,intervention_name,disease,outcome_measures,outcome_timeframes,durationMonths
0,NCT03202394,COMPLETED,['PHASE2'],"This is a randomized, double blind, placebo co...",Inclusion Criteria:\n\n1. Has provided (or rel...,:\n\n1. Age \< 18 years or \>75 years old\n2. ...,"['DRUG', 'DRUG']","['BIO-11006', 'Placebo']","['Respiratory Distress Syndrome, Adult']",['Incidence of treatment-emergent adverse even...,['28 days'],32.266667
1,NCT02138214,COMPLETED,['PHASE2'],PRIMARY OBJECTIVES:\n\nI. To determine the rat...,Inclusion Criteria:\n\n* Pre-operative diagnos...,:\n\n* Largest papillary thyroid carcinoma \< ...,"['PROCEDURE', 'PROCEDURE', 'OTHER']","['Thyroidectomy', 'entral lymph node dissectio...","['Stage I Papillary Thyroid Cancer', 'Stage II...",['Percentage of Participants With Transient Hy...,"['Post-operative day 1', 'At day 12', '2 weeks...",65.733333
2,NCT00976274,COMPLETED,['NA'],The details of study objective are followed by...,Inclusion Criteria:(three or more of following...,: (any one of following factors)\n\n* uncontro...,"['DIETARY_SUPPLEMENT', 'DIETARY_SUPPLEMENT']","['Korean red ginseng', 'starch']",['Metabolic Syndrome'],['Change in the Pre- and Post-treatment Systol...,['baseline and 12 weeks'],17.266667
3,NCT05529719,COMPLETED,['NA'],Objective: This study evaluates the effect of ...,Inclusion Criteria:\n\n* 18 to 30 years of age...,:\n\n* Volunteers who did not meet the inclusi...,['OTHER'],['Clinical Pilates Exercises'],['Low Back Pain'],['Prim Outcome - Oswestry Dysability Index'],['12 weeks'],25.266667
4,NCT02323191,COMPLETED,['PHASE1'],"This Phase 1, open-label, multicenter, global ...",Inclusion Criteria:\n\n* Eastern Cooperative O...,\n* Measurable disease at baseline as per RECI...,"['DRUG', 'DRUG']","['Atezolizumab', 'Emactuzumab']",['Solid Cancers'],['Percentage of Participants With Dose Limitin...,"['21 days', '21 days', 'Baseline up to 3 years']",68.033333
...,...,...,...,...,...,...,...,...,...,...,...,...
184620,NCT05883852,RECRUITING,['PHASE3'],The objective of this study is to conduct a ra...,Inclusion Criteria:\n\n* Women aged 18-70；\n* ...,:\n\n* Bilateral breast cancer or carcinoma in...,"['DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG']","['Docetaxel', 'carboplatin', 'Trastuzumab', 'P...",['HER2 Positive Early Breast Cancer'],['iDFS'],['5 years'],98.200000
184621,NCT02180724,ACTIVE_NOT_RECRUITING,['PHASE2'],Clinical studies have shown that targeting the...,Inclusion Criteria:\n\n1. Men and women ≥18 ye...,":\n\n1. Prior malignancy, except for adequatel...","['DRUG', 'DRUG']","['Acalabrutinib (ACP-196)', 'Acalabrutinib (AC...",['Waldenström Macroglobulinemia (WM)'],['Overall Response Rate (ORR) of Acalabrutinib...,['Up to approximately 3.8 years. Data cut at w...,61.633333
184622,NCT00487305,ACTIVE_NOT_RECRUITING,['PHASE1'],* The dose of vaccine will depend upon how man...,Inclusion Criteria:\n\n* Histologically confir...,:\n\n* Uncontrolled active infection or illnes...,['BIOLOGICAL'],['Lethally Irradiated Lymphoma cells with GM-C...,['Follicular Lymphoma'],['To determine the safety and toxicity of admi...,['2 years'],195.800000
184623,NCT05786924,RECRUITING,['PHASE1'],"BDTX-4933-101 is a first-in-human, open-label,...",Key Inclusion Criteria:\n\n1. Disease criteria...,:\n\n1. Cancer that has a known MEK1/2 mutatio...,['DRUG'],['BDTX-4933'],"['Non-small Cell Lung Cancer', 'Histiocytic Ne...",['Dose Escalation: Incidence of dose-limiting ...,"['The first 28-day cycle (Cycle 1)', 'Day 1 of...",38.000000


In [40]:
# Merge together with clinical trial study information dataframe
incompleted_preds_df = pd.read_csv("../results/incompleted_preds_df.csv")


study_info_df = pd.merge(
    study_info_df, incompleted_preds_df, on="nctId", how="left", suffixes=("", "_y")
)


study_info_df = study_info_df.drop("durationMonths_y", axis=1)
study_info_df

,nctId,status,phases,description,inclusion_criteria,exclusion_criteria,intervention_type,intervention_name,disease,outcome_measures,outcome_timeframes,durationMonths,predicted_durationMonths
0,NCT03202394,COMPLETED,['PHASE2'],"This is a randomized, double blind, placebo co...",Inclusion Criteria:\n\n1. Has provided (or rel...,:\n\n1. Age \< 18 years or \>75 years old\n2. ...,"['DRUG', 'DRUG']","['BIO-11006', 'Placebo']","['Respiratory Distress Syndrome, Adult']",['Incidence of treatment-emergent adverse even...,['28 days'],32.266667,NaN
1,NCT02138214,COMPLETED,['PHASE2'],PRIMARY OBJECTIVES:\n\nI. To determine the rat...,Inclusion Criteria:\n\n* Pre-operative diagnos...,:\n\n* Largest papillary thyroid carcinoma \< ...,"['PROCEDURE', 'PROCEDURE', 'OTHER']","['Thyroidectomy', 'entral lymph node dissectio...","['Stage I Papillary Thyroid Cancer', 'Stage II...",['Percentage of Participants With Transient Hy...,"['Post-operative day 1', 'At day 12', '2 weeks...",65.733333,NaN
2,NCT00976274,COMPLETED,['NA'],The details of study objective are followed by...,Inclusion Criteria:(three or more of following...,: (any one of following factors)\n\n* uncontro...,"['DIETARY_SUPPLEMENT', 'DIETARY_SUPPLEMENT']","['Korean red ginseng', 'starch']",['Metabolic Syndrome'],['Change in the Pre- and Post-treatment Systol...,['baseline and 12 weeks'],17.266667,NaN
3,NCT05529719,COMPLETED,['NA'],Objective: This study evaluates the effect of ...,Inclusion Criteria:\n\n* 18 to 30 years of age...,:\n\n* Volunteers who did not meet the inclusi...,['OTHER'],['Clinical Pilates Exercises'],['Low Back Pain'],['Prim Outcome - Oswestry Dysability Index'],['12 weeks'],25.266667,NaN
4,NCT02323191,COMPLETED,['PHASE1'],"This Phase 1, open-label, multicenter, global ...",Inclusion Criteria:\n\n* Eastern Cooperative O...,\n* Measurable disease at baseline as per RECI...,"['DRUG', 'DRUG']","['Atezolizumab', 'Emactuzumab']",['Solid Cancers'],['Percentage of Participants With Dose Limitin...,"['21 days', '21 days', 'Baseline up to 3 years']",68.033333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
184620,NCT05883852,RECRUITING,['PHASE3'],The objective of this study is to conduct a ra...,Inclusion Criteria:\n\n* Women aged 18-70；\n* ...,:\n\n* Bilateral breast cancer or carcinoma in...,"['DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG']","['Docetaxel', 'carboplatin', 'Trastuzumab', 'P...",['HER2 Positive Early Breast Cancer'],['iDFS'],['5 years'],98.200000,48.601350
184621,NCT02180724,ACTIVE_NOT_RECRUITING,['PHASE2'],Clinical studies have shown that targeting the...,Inclusion Criteria:\n\n1. Men and women ≥18 ye...,":\n\n1. Prior malignancy, except for adequatel...","['DRUG', 'DRUG']","['Acalabrutinib (ACP-196)', 'Acalabrutinib (AC...",['Waldenström Macroglobulinemia (WM)'],['Overall Response Rate (ORR) of Acalabrutinib...,['Up to approximately 3.8 years. Data cut at w...,61.633333,26.629488
184622,NCT00487305,ACTIVE_NOT_RECRUITING,['PHASE1'],* The dose of vaccine will depend upon how man...,Inclusion Criteria:\n\n* Histologically confir...,:\n\n* Uncontrolled active infection or illnes...,['BIOLOGICAL'],['Lethally Irradiated Lymphoma cells with GM-C...,['Follicular Lymphoma'],['To determine the safety and toxicity of admi...,['2 years'],195.800000,61.075123
184623,NCT05786924,RECRUITING,['PHASE1'],"BDTX-4933-101 is a first-in-human, open-label,...",Key Inclusion Criteria:\n\n1. Disease criteria...,:\n\n1. Cancer that has a known MEK1/2 mutatio...,['DRUG'],['BDTX-4933'],"['Non-small Cell Lung Cancer', 'Histiocytic Ne...",['Dose Escalation: Incidence of dose-limiting ...,"['The first 28-day cycle (Cycle 1)', 'Day 1 of...",38.000000,50.798630


In [41]:
# Make copy for reference in Dashboard output so as not to clutter the DataTable with long strings such as the descriptions and eligibility criteria and remove from original
description_keyword_df = study_info_df.copy()[
    [
        "nctId",
        "status",
        "durationMonths",
        "predicted_durationMonths",
        "description",
        "disease",
        "intervention_type",
        "intervention_name",
    ]
]

study_info_df = study_info_df[
    [
        "nctId",
        "status",
        "durationMonths",
        "predicted_durationMonths",
        "disease",
        "intervention_type",
        "intervention_name",
    ]
]

study_info_df

,nctId,status,durationMonths,predicted_durationMonths,disease,intervention_type,intervention_name
0,NCT03202394,COMPLETED,32.266667,NaN,"['Respiratory Distress Syndrome, Adult']","['DRUG', 'DRUG']","['BIO-11006', 'Placebo']"
1,NCT02138214,COMPLETED,65.733333,NaN,"['Stage I Papillary Thyroid Cancer', 'Stage II...","['PROCEDURE', 'PROCEDURE', 'OTHER']","['Thyroidectomy', 'entral lymph node dissectio..."
2,NCT00976274,COMPLETED,17.266667,NaN,['Metabolic Syndrome'],"['DIETARY_SUPPLEMENT', 'DIETARY_SUPPLEMENT']","['Korean red ginseng', 'starch']"
3,NCT05529719,COMPLETED,25.266667,NaN,['Low Back Pain'],['OTHER'],['Clinical Pilates Exercises']
4,NCT02323191,COMPLETED,68.033333,NaN,['Solid Cancers'],"['DRUG', 'DRUG']","['Atezolizumab', 'Emactuzumab']"
...,...,...,...,...,...,...,...
184620,NCT05883852,RECRUITING,98.200000,48.601350,['HER2 Positive Early Breast Cancer'],"['DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG']","['Docetaxel', 'carboplatin', 'Trastuzumab', 'P..."
184621,NCT02180724,ACTIVE_NOT_RECRUITING,61.633333,26.629488,['Waldenström Macroglobulinemia (WM)'],"['DRUG', 'DRUG']","['Acalabrutinib (ACP-196)', 'Acalabrutinib (AC..."
184622,NCT00487305,ACTIVE_NOT_RECRUITING,195.800000,61.075123,['Follicular Lymphoma'],['BIOLOGICAL'],['Lethally Irradiated Lymphoma cells with GM-C...
184623,NCT05786924,RECRUITING,38.000000,50.798630,"['Non-small Cell Lung Cancer', 'Histiocytic Ne...",['DRUG'],['BDTX-4933']


## Dashboard Creation

In [51]:
# Create separate "id" value as the index to keep the rows and indexing consistent through sorting and filtering
study_info_df["id"] = study_info_df["nctId"]
study_info_df.set_index("id", inplace=True, drop=False)

app = Dash(__name__)

# Text color settings for Dashboard output.
# Change to 'white' (or any other acceptable color) if using a different background color (such as dark mode making 'black' text hard to see)
color = "black"
# color = 'white'

app.layout = html.Div(
    [
        html.H1(
            "TrialDuraPredict Clinical Trials Dashboard",
            style={"color": color},
        ),
        html.H1(
            "The TrialDuraPredict dashboard was made to help show and filter information on the thousands of clinical trials we examined. Below are some functions to get you started exploring:",
            style={"fontSize": 20, "color": color},
        ),
        html.H1(
            """
                - Filtering: You can filter the column values by typing in the top row below the column title. For example, you can filter under the "status" column "rec" to find all the studies in the
                'RECRUITING' and 'ACTIVE_NOT_RECRUITING' phases instead of completed trials. You can further filter using mutltiple columns. Try filtering diseases to look for "heart" related studies!
                To undo any filtering, simply delete the text in the column(s) and press "Enter" again.
                """,
            style={"fontSize": 20, "color": color},
        ),
        html.H1(
            """
                - Sorting: You can sort the column values clicking the up and down arrows to the left of the column title. Be careful not to delete the column by clicking the trashcan!
                To undo any sorting, simply click twice more to invert the sorting then set it back to normal.
                """,
            style={"fontSize": 20, "color": color},
        ),
        html.H1(
            """
                - Selecting a Cell: By clicking on any cell, the associated study description and trial duration will be displayed below the DataTable, as well as update the visualization.
                """,
            style={"fontSize": 20, "color": color},
        ),
        html.H1(
            """
                - Visualization: The bar chart below compares the true trial duration (blue) and predicted trial duration (red)
                of all filtered studies and calculates the true mean duration as shown by the black line.
                With too many datapoints, this will not be shown, so try filtering first!
                Hover over this over the individual bars for more information and scroll to zoom in and out, dragging to stay focused around 0-100 months.
                Selecting a cell will highlight the study in the bar chart for comparison.
                """,
            style={"fontSize": 20, "color": color},
        ),
        html.H1(
            "Trial Information",
            style={"color": color},
        ),
        dash_table.DataTable(
            id="datatable-row-ids",
            columns=[
                {"name": i, "id": i, "deletable": True}
                for i in study_info_df.columns
                # omit the id column
                if i != "id"
            ],
            data=study_info_df.to_dict("records"),
            editable=True,
            filter_action="native",
            filter_options={
                "placeholder_text": "Filter column...",
                "case": "insensitive",
            },
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            row_deletable=True,
            selected_rows=[],
            page_action="native",
            page_current=0,
            page_size=10,
        ),
        html.H1(
            "Study Description",
            style={"color": color},
        ),
        html.Div(
            id="datatable-row-ids-container",
            style={"fontSize": 24, "color": color},
        ),
        html.H1(
            "Trial Duration",
            style={"color": color},
        ),
        html.Div(
            id="trial-dura-text",
            style={"fontSize": 24, "color": color},
        ),
        html.H1(
            "Trial Duration by Study NCTID",
            style={"color": color},
        ),
        # Use the 'dvc' library to allow displaying Altair charts in Plotly Dash
        dvc.Vega(id="altair-chart", opt={"renderer": "svg", "actions": False}),
    ]
)


# Update DataTable based on filtering and setup the study information to display as text based on this
@callback(
    Output("datatable-row-ids-container", "children"),
    Output("trial-dura-text", "children"),
    Input("datatable-row-ids", "derived_virtual_row_ids"),
    Input("datatable-row-ids", "selected_row_ids"),
    Input("datatable-row-ids", "active_cell"),
)
def update_graphs(row_ids, selected_row_ids, active_cell):
    if row_ids is None:
        row_ids = study_info_df["id"]

    # setting nctId variable
    active_row_id = active_cell["row_id"] if active_cell else None

    # display study description below datatable
    description = (
        description_keyword_df.loc[
            description_keyword_df.nctId == active_row_id, "description"
        ].item()
        if active_cell
        else "[Study Description Goes Here]"
    )
    # display true/current study duration below datatable
    dura = (
        str(
            description_keyword_df.loc[
                description_keyword_df.nctId == active_row_id, "durationMonths"
            ].item()
        )
        + " months"
        if active_cell
        else "[Trial Duration Goes Here]"
    )

    return description, dura


# Update the Bar Chart based on how the table is filtered and sorted, as well as if any cells are selected.
@callback(
    Output("altair-chart", "spec"),
    Input("datatable-row-ids", "derived_virtual_row_ids"),
    Input("datatable-row-ids", "selected_row_ids"),
    Input("datatable-row-ids", "active_cell"),
)
def update_chart(row_ids, selected_row_ids, active_cell):
    # Keep row ID's even when filtering and sorting by basing them on NCTID
    if row_ids is None:
        dff = study_info_df.drop_duplicates("id")
        row_ids = study_info_df["id"]
    else:
        dff = study_info_df.loc[row_ids].drop_duplicates("id")

    # Update text output and chart generate chart based on select cell, if one is selected
    if active_cell:
        # Actual Duration Chart
        chart = (
            alt.Chart(dff)
            .mark_bar()
            .encode(
                alt.X("nctId", sort=None),
                y="durationMonths",
                tooltip=list(study_info_df.columns),
                opacity=alt.condition(
                    alt.datum.nctId == active_cell["row_id"],
                    alt.value(0.7),
                    alt.value(0.3),
                ),
            )
            .interactive()
        )

        # Predicted Duration Chart
        chart2 = (
            alt.Chart(dff)
            .mark_bar(color="darkred")
            .encode(
                alt.X("nctId", sort=None),
                y="predicted_durationMonths",
                tooltip=list(study_info_df.columns),
                opacity=alt.condition(
                    alt.datum.nctId == active_cell["row_id"],
                    alt.value(0.7),
                    alt.value(0.3),
                ),
            )
            .interactive()
        )

    # Update if no cell is selected
    else:
        chart = (
            alt.Chart(dff)
            .mark_bar(opacity=0.7)
            .encode(
                alt.X("nctId", sort=None),
                y="durationMonths",
                tooltip=list(study_info_df.columns),
            )
            .interactive()
        )

        chart2 = (
            alt.Chart(dff)
            .mark_bar(color="darkred", opacity=0.7)
            .encode(
                alt.X("nctId", sort=None),
                y="predicted_durationMonths",
                tooltip=list(study_info_df.columns),
            )
            .interactive()
        )

    # True/current trial duration mean line generation
    line = (
        alt.Chart(dff)
        .mark_rule(color="black")
        .encode(
            y="mean(durationMonths):Q",
            size=alt.value(5),
            tooltip="mean(durationMonths):Q",
        )
    )

    # Altair only shows up to 5000 rows for data, which we could increase, but we do not need to see the entire dataset on a single barchart
    try:
        return (
            (chart + chart2 + line)
            .encode(y=alt.Y(title="Trial Duration (Months)"))
            .configure_axis(labelFontSize=14, titleFontSize=18)
            .to_dict()
        )
    except:
        return None


if __name__ == "__main__":
    app.run(debug=True)

C:\Users\97dan\AppData\Local\Temp\ipykernel_22804\173766365.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

